# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
from pprint import pprint

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [50]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(cities)
len(cities)

['atuona', 'makrikapa', 'vaini', 'punta arenas', 'ribeira grande', 'kruisfontein', 'ushuaia', 'albany', 'solnechnyy', 'sabzevar', 'ilulissat', 'girona', 'saint-joseph', 'dodola', 'puerto leguizamo', 'iqaluit', 'avera', 'severo-kurilsk', 'sur', 'cherskiy', 'half moon bay', 'abu zabad', 'rikitea', 'kaitangata', 'ponta do sol', 'seoul', 'sorland', 'torbay', 'avarua', 'dikson', 'hobart', 'busselton', 'saint george', 'srednekolymsk', 'sinnamary', 'port elizabeth', 'saskylakh', 'karkaralinsk', 'barrow', 'kulunda', 'yellowknife', 'beringovskiy', 'east london', 'shenjiamen', 'olinda', 'caravelas', 'belushya guba', 'barentsburg', 'coahuayana', 'cordoba', 'bredasdorp', 'bowen', 'taolanaro', 'walvis bay', 'cedar city', 'kieta', 'tuktoyaktuk', 'beberibe', 'aurangabad', 'vila velha', 'new norfolk', 'hilo', 'yomitan', 'lisala', 'kitimat', 'tigil', 'chuy', 'mar del plata', 'saint-philippe', 'vanavara', 'aras', 'plouzane', 'taoudenni', 'san patricio', 'kargil', 'abu samrah', 'mandalgovi', 'georgetown'

643

In [51]:
city_list = cities
city_list

['atuona',
 'makrikapa',
 'vaini',
 'punta arenas',
 'ribeira grande',
 'kruisfontein',
 'ushuaia',
 'albany',
 'solnechnyy',
 'sabzevar',
 'ilulissat',
 'girona',
 'saint-joseph',
 'dodola',
 'puerto leguizamo',
 'iqaluit',
 'avera',
 'severo-kurilsk',
 'sur',
 'cherskiy',
 'half moon bay',
 'abu zabad',
 'rikitea',
 'kaitangata',
 'ponta do sol',
 'seoul',
 'sorland',
 'torbay',
 'avarua',
 'dikson',
 'hobart',
 'busselton',
 'saint george',
 'srednekolymsk',
 'sinnamary',
 'port elizabeth',
 'saskylakh',
 'karkaralinsk',
 'barrow',
 'kulunda',
 'yellowknife',
 'beringovskiy',
 'east london',
 'shenjiamen',
 'olinda',
 'caravelas',
 'belushya guba',
 'barentsburg',
 'coahuayana',
 'cordoba',
 'bredasdorp',
 'bowen',
 'taolanaro',
 'walvis bay',
 'cedar city',
 'kieta',
 'tuktoyaktuk',
 'beberibe',
 'aurangabad',
 'vila velha',
 'new norfolk',
 'hilo',
 'yomitan',
 'lisala',
 'kitimat',
 'tigil',
 'chuy',
 'mar del plata',
 'saint-philippe',
 'vanavara',
 'aras',
 'plouzane',
 'taoude

In [71]:
df = pd.DataFrame(city_list)
df["City"]=cities
df = df.drop([1])
df

,0,City
0,atuona,atuona
2,vaini,vaini
3,punta arenas,punta arenas
4,ribeira grande,ribeira grande
5,kruisfontein,kruisfontein
...,...,...
638,masallatah,masallatah
639,cap-aux-meules,cap-aux-meules
640,muzhi,muzhi
641,cayenne,cayenne


In [72]:
city_short_df = df.head(10)
city_short_df 

,0,City
0,atuona,atuona
2,vaini,vaini
3,punta arenas,punta arenas
4,ribeira grande,ribeira grande
5,kruisfontein,kruisfontein
6,ushuaia,ushuaia
7,albany,albany
8,solnechnyy,solnechnyy
9,sabzevar,sabzevar
10,ilulissat,ilulissat


In [73]:
# Creating small set to execute loops
# url = "http://api.openweathermap.org/data/2.5/weather?"
# units = "imperial"


# lat = []
# lng = []
# temps = []
# humidity = []
# cloudiness = []
# windspeed = []


# # looping through test cities:
# # for index, row in city_short_df.iterrows():
# for index, row in city_short_df.iterrows():
#     city = row["City"]
#     weather_url = f"{url}appid={weather_api_key}&units={units}&q={city}"
#     response = requests.get(weather_url).json()

    
#     try:
        
#         city_short_df.loc[index, "Lng"] = lng.append(response["coord"]["lon"])
#         city_short_df.loc[index, "Temp"] = temps.append(response["main"]["temp_max"])
#         city_short_df.loc[index, "Humidity"] = humidity.append(response["main"]["humidity"])
#         city_short_df.loc[index, "Cloudiness"] = cloudiness.append(response["clouds"]["all"])
#         city_short_df.loc[index, "WindSpeed"] = windspeed.append(response["wind"]["speed"])
#         city_short_df.loc[index, "Lat"] = lat.append(response["coord"]["lat"])
#         print(f"processing {index}| {city}")
        
#     except KeyError:
#         print("skipping {city}")


/Users/Tyson/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Tyson/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


processing 0| atuona
processing 2| vaini
processing 3| punta arenas
processing 4| ribeira grande
processing 5| kruisfontein
processing 6| ushuaia
processing 7| albany
processing 8| solnechnyy
processing 9| sabzevar
processing 10| ilulissat


In [74]:
# print(lat)
# print(lng)
# print(temps)
# print(humidity)
# print(cloudiness)

[-9.8, -21.2, -53.15, 38.52, -34, -54.8, 42.6, 50.72, 36.21, 69.22]
[-139.03, -175.2, -70.92, -28.7, 24.73, -68.3, -73.97, 136.63, 57.68, -51.1]
[78.4, 77, 44.6, 61.41, 70, 48.2, 34, 29.32, 60.8, 35.6]
[79, 83, 93, 66, 83, 57, 78, 83, 25, 74]
[16, 75, 90, 52, 100, 75, 11, 36, 0, 100]


In [77]:
# # Creating small dataframe to test
# citieslis = {"Lat": [lat], "Lng": [lng], "Temp": [temps],
#              "Humidity": [humidity], "Cloudiness":[cloudiness],
#              "windSpeed": [windspeed]}
# city_short_df["Lat"] = lat
# city_short_df["Lng"] = lng
# city_short_df["Temps"] = temps
# city_short_df["Humidity"] = humidity
# city_short_df["Cloudiness"] = cloudiness
# city_short_df["WindSpeed"] = windspeed
# city_short_df

In [79]:
# actual data set
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"


lat = []
lng = []
temps = []
humidity = []
cloudiness = []
windspeed = []


# looping through test cities:
# for index, row in city_short_df.iterrows():
for index, row in df.iterrows():
    city = row["City"]
    weather_url = f"{url}appid={weather_api_key}&units={units}&q={city}"
    response = requests.get(weather_url).json()

    
    try:
        
        df.loc[index, "Lng"] = lng.append(response["coord"]["lon"])
        df.loc[index, "Temp"] = temps.append(response["main"]["temp_max"])
        df.loc[index, "Humidity"] = humidity.append(response["main"]["humidity"])
        df.loc[index, "Cloudiness"] = cloudiness.append(response["clouds"]["all"])
        df.loc[index, "WindSpeed"] = windspeed.append(response["wind"]["speed"])
        df.loc[index, "Lat"] = lat.append(response["coord"]["lat"])
        print(f"processing {index}| {city}")
        
    except KeyError:
        print("skipping {city}")
        df.drop(index, inplace=True)


processing 0| atuona
processing 2| vaini
processing 3| punta arenas
processing 4| ribeira grande
processing 5| kruisfontein
processing 6| ushuaia
processing 7| albany
processing 8| solnechnyy
processing 9| sabzevar
processing 10| ilulissat
processing 11| girona
processing 12| saint-joseph
processing 13| dodola
processing 14| puerto leguizamo
processing 15| iqaluit
processing 16| avera
processing 17| severo-kurilsk
processing 18| sur
processing 19| cherskiy
processing 20| half moon bay
processing 21| abu zabad
processing 22| rikitea
processing 23| kaitangata
processing 24| ponta do sol
processing 25| seoul
processing 26| sorland
processing 27| torbay
processing 28| avarua
processing 29| dikson
processing 30| hobart
processing 31| busselton
processing 32| saint george
processing 33| srednekolymsk
processing 34| sinnamary
processing 35| port elizabeth
processing 36| saskylakh
skipping {city}
processing 38| barrow
processing 39| kulunda
processing 40| yellowknife
processing 41| beringovski

processing 339| prince rupert
processing 340| podosinovets
processing 341| kaman
processing 342| russellville
processing 343| talnakh
processing 344| bonavista
processing 345| port hardy
processing 346| cabo san lucas
skipping {city}
processing 348| murgab
processing 349| provideniya
processing 350| hami
processing 351| elban
skipping {city}
processing 353| sisimiut
processing 354| riyadh
processing 355| kungurtug
processing 356| waingapu
processing 357| bilma
processing 358| pyshchug
processing 359| konstantinovka
processing 360| dalbandin
processing 361| pangnirtung
processing 362| khorinsk
processing 363| marienburg
processing 364| biltine
processing 365| bani walid
processing 366| samarai
processing 367| saint-augustin
processing 368| souillac
processing 369| liusha
processing 370| tevaitoa
processing 371| lipari
processing 372| bambous virieux
processing 373| gouyave
processing 374| qaqortoq
processing 375| khatanga
processing 376| imbituba
processing 377| saint-francois
processin

In [82]:
df

,0,City,Lng,Temp,Humidity,Cloudiness,WindSpeed,Lat
0,atuona,atuona,NaN,NaN,NaN,NaN,NaN,NaN
2,vaini,vaini,NaN,NaN,NaN,NaN,NaN,NaN
3,punta arenas,punta arenas,NaN,NaN,NaN,NaN,NaN,NaN
4,ribeira grande,ribeira grande,NaN,NaN,NaN,NaN,NaN,NaN
5,kruisfontein,kruisfontein,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
638,masallatah,masallatah,NaN,NaN,NaN,NaN,NaN,NaN
639,cap-aux-meules,cap-aux-meules,NaN,NaN,NaN,NaN,NaN,NaN
640,muzhi,muzhi,NaN,NaN,NaN,NaN,NaN,NaN
641,cayenne,cayenne,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
# Creating small dataframe to test
citieslis = {"Lat": [lat], "Lng": [lng], "Temp": [temps],
             "Humidity": [humidity], "Cloudiness":[cloudiness],
             "windSpeed": [windspeed]}
df["Lat"] = lat
df["Lng"] = lng
df["Temps"] = temps
df["Humidity"] = humidity
df["Cloudiness"] = cloudiness
df["WindSpeed"] = windspeed
# city_short_df
df

,0,City,Lng,Temp,Humidity,Cloudiness,WindSpeed,Lat,Temps
0,atuona,atuona,-139.03,NaN,79,16,21.32,-9.80,78.40
2,vaini,vaini,-175.20,NaN,88,75,12.75,-21.20,75.20
3,punta arenas,punta arenas,-70.92,NaN,93,90,12.75,-53.15,44.60
4,ribeira grande,ribeira grande,-28.70,NaN,66,52,12.30,38.52,61.41
5,kruisfontein,kruisfontein,24.73,NaN,81,100,1.99,-34.00,71.01
...,...,...,...,...,...,...,...,...,...
638,masallatah,masallatah,14.00,NaN,66,1,8.16,32.62,64.06
639,cap-aux-meules,cap-aux-meules,-61.86,NaN,74,91,20.80,47.38,28.40
640,muzhi,muzhi,64.67,NaN,86,98,16.08,65.37,36.32
641,cayenne,cayenne,-52.33,NaN,100,75,1.12,4.93,77.00


In [86]:
print(lat)


[-9.8, -21.2, -53.15, 38.52, -34, -54.8, 42.6, 50.72, 36.21, 69.22, 41.98, -21.37, 6.98, -0.19, 63.75, 33.19, 50.68, 22.57, 68.75, 37.46, 12.35, -23.12, -46.28, 32.67, 37.57, 67.67, 47.67, -21.21, 73.51, -42.88, -33.65, 37.1, 67.45, 5.38, -33.92, 71.92, 71.29, 52.57, 62.46, 63.05, -33.02, 29.96, -8.01, -17.71, 18.73, -31.41, -34.53, -20.02, -22.96, 37.68, -6.22, 69.45, -4.18, 19.88, -20.33, -42.78, 19.73, 2.15, 54.05, 57.8, -33.7, -38, -21.36, 60.34, 42.56, 48.38, 22.68, 28.02, 34.57, 35.3, 45.76, 5.41, 29.03, 65.7, -33.59, 55.74, -34.48, -16.7, 36.21, -34.42, -46.6, 42.78, 51.68, -33.87, 60.14, 51.32, -3.37, 66.71, -14.97, 54.77, 64.5, 47.91, 62.08, 43.8, -32.25, -0.74, 70.63, 18.39, -3.77, 27.72, 32.7, 22.08, -0.18, 38.73, 77.48, 8.12, -4.06, 11.78, 29.27, -32.03, 71.69, 42.1, -20.41, -8.47, 40.28, 14.72, -0.6, 34.67, 44.9, -46.19, 29.6, 42.42, 45.02, 20.93, 65.97, -24.79, 3.78, -6.28, 16.52, 50.15, -8.9, 28.66, 37.42, 12.18, -6.37, 13.22, -37.62, 66.83, 21.46, 28.5, 35.02, 7.42, 30.

# temps = cities_df["Max Temp"]
humidity = cities_df["Humidity"]
cloudiness = cities_df["Cloudiness"]
lat = cities_df["Lat"]
windspeed = cities_df["Wind Speed"]

print(north_lat)

In [ ]:
# North Data
n_temps = north_df["Max Temp"]
n_humidity = north_df["Humidity"]
n_cloudiness = north_df["Cloudiness"]
n_windspeed = north_df["Wind Speed"]
n_lat = north_df["Lat"]

# South Data
s_temps = south_df["Max Temp"]
s_humidity = south_df["Humidity"]
s_cloudiness = south_df["Cloudiness"]
s_windspeed = south_df["Wind Speed"]
s_lat = south_df["Lat"]

In [ ]:
plt.scatter(temps, lat)
plt.title("Temp Vs Latitude")
plt.xlabel("Temperature")
plt.ylabel("Latitude")

print("As the latitude reaches the Equator, the tempurature gets warmer")

In [ ]:
plt.scatter(humidity, lat)
plt.title("Humidity Vs Latitude")
plt.xlabel("Humidity")
plt.ylabel("Latitude")

In [ ]:
plt.scatter(cloudiness, lat)
plt.title("Cloudiness Vs Latitude")
plt.xlabel("Cloudiness %")
plt.ylabel("Latitude")

In [ ]:
plt.scatter(windspeed, lat)
plt.title("WindSpeed Vs Latitude")
plt.xlabel("Windspeed (mph)")
plt.ylabel("Latitude")

In [ ]:
# Temperature (F) vs. Latitude
# Northern Hem
plt.subplot(2,2.5,1)
plt.scatter(n_temps, north_lat)
plt.title("Temp Vs Latitude- Northern Hem")


# # Southern Hem
plt.subplot(2,3,3)
plt.scatter(s_temps, south_lat)
plt.title("Temp Vs Latitude- Southern Hem")



In [ ]:
north_df = cities_df.loc[cities_df["Lat"] >0,:]
# north_df
south_df = cities_df.loc[cities_df["Lat"] <0,:]
north_df

In [ ]:
# Humidity vs. Latitude
# Northern Hem
plt.subplot(2,2.5,1)
plt.scatter(n_humidity, north_lat)
plt.title("Temp Vs Latitude- Northern Hem")


# # Southern Hem
plt.subplot(2,3,3)
plt.scatter(s_humidity, south_lat)
plt.title("Temp Vs Latitude- Southern Hem")

In [ ]:
# Cloudiness vs. Latitude
# Northern Hem
plt.subplot(2,2.5,1)
plt.scatter(n_cloudiness, north_lat)
plt.title("Temp Vs Latitude- Northern Hem")


# # Southern Hem
plt.subplot(2,3,3)
plt.scatter(s_cloudiness, south_lat)
plt.title("Temp Vs Latitude- Southern Hem")

In [ ]:
# Wind Speed vs. Latitude
# Northern Hem
plt.subplot(2,2.5,1)
plt.scatter(n_windspeed, north_lat)
plt.title("Temp Vs Latitude- Northern Hem")


# # Southern Hem
plt.subplot(2,3,3)
plt.scatter(s_windspeed, south_lat)
plt.title("Temp Vs Latitude- Southern Hem")

## Generate Cities List

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression